In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from src.accelerometer import plot_fourier_transformation, plot_acceleration, plot_feature_columns, accelerometer_feature_engineering
from src.kmeans import kmeans
from src.principal_component_analysis import principal_component_analysis, plot_principal_component_analysis
from src.decision_tree import decision_tree
from src.device_handling import split_by_device
from src.plotting import box_plot_columns
from src.time_series import align_sequences_to_same_length, median_filter, run_time_series_algorithms
from src.ml_util import run_feature_algorithms
from src.pandas_util import correlation_matrix,extract_sample_sessions,extract_subject_dataframes

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
drift_df = pd.read_csv('drift_accelerations.csv')
drift_df = drift_df[drift_df['age_group']!=0]

In [ ]:
drift_df.head(5)

In [ ]:
drift_df.describe()

In [ ]:
drift_df = median_filter(drift_df)

In [ ]:
def clean_accelerations(df):
    accelerations_copy_df = df[(df['duration'] > 5000)] 
    accelerations_copy_df = accelerations_copy_df[(accelerations_copy_df['duration'] < 35000)] 
    return accelerations_copy_df
drift_df = clean_accelerations(drift_df)

In [ ]:
single_session_30_df, single_session_50_df = extract_sample_sessions(drift_df,subject_index=3, session_index=0)
session_30_device_split = split_by_device(single_session_30_df)
session_50_device_split = split_by_device(single_session_50_df)
plot_acceleration(session_30_device_split, title=['dominant age-group \'<30\'', 'non-dominant age-group \'<30\''], subplots=False)
plot_acceleration(session_50_device_split, title=['dominant age-group \'>50\'', 'non-dominant age-group \'>50\''], subplots=False)

In [ ]:
split_by_device(single_session_50_df)[0]['hand'].unique()

In [ ]:
subject_30_df, subject_50_df = extract_subject_dataframes(drift_df)
plot_acceleration(split_by_device(subject_30_df), title=['dominant age-group \'<30\'', 'non-dominant age-group \'<30\''], subplots=False)
plot_acceleration(split_by_device(subject_50_df), title=['dominant age-group \'>50\'', 'non-dominant age-group \'>50\''], subplots=False)

In [ ]:
plot_fourier_transformation(single_session_30_df, 'test session')

In [ ]:
box_plot_columns(single_session_30_df)

In [ ]:
drift_filtered_df = pd.DataFrame(columns=drift_df.columns)
for device in drift_df['device'].unique():
    device_df = drift_df[drift_df['device']==device]
    device_filtered_df = median_filter(device_df)
    drift_filtered_df = pd.concat([drift_filtered_df, device_filtered_df])
drift_filtered_df.head()

In [ ]:
drift_processed_df = accelerometer_feature_engineering(drift_filtered_df)
drift_processed_df = drift_processed_df.reset_index(drop=False)
drift_processed_df = drift_processed_df.drop('hand', axis=1)
drift_processed_df = drift_processed_df.drop('device', axis=1)
drift_processed_df.head()

In [ ]:
def drift_features(df):
    entries = []
    for uuid in df['uuid'].unique():
        current_df = df[df['uuid'] == uuid]
        entry = {}
        entry['age_group'] = current_df['age_group'].unique().max()
        entry['subject'] = current_df['subject'].unique().any()
        entry['uuid'] = current_df['uuid'].unique().any()
        entry['file'] = current_df['file'].unique().any()
        entry['duration'] = current_df['duration'].max()
        session_devices = split_by_device(current_df)
        device_count = 0
        for device_df in session_devices:
            device_processed_df = accelerometer_feature_engineering(device_df)
            device_processed_df = device_processed_df.drop(columns=['x_sem', 'y_sem', 'z_sem', 'mag_sem','duration'])
            suffix = f'_{device_count}'
            device_count += 1
            device_processed_df.columns += suffix
            for column in device_processed_df.columns:
                entry[column]=device_processed_df[column].max()
        entries.append(entry)
    return pd.DataFrame(entries)

In [ ]:
def calc_device_diffs(df, columns):
    local_df = df.copy()
    new_suffix = '_diff'
    processed_columns = []
    for column in columns:
        column_without_suffix = column.removesuffix('_0').removesuffix('_1')
        if column_without_suffix not in processed_columns:
            processed_columns.append(column_without_suffix)
            new_column = column_without_suffix + new_suffix
            column_device_1 = column_without_suffix+'_0'
            column_device_2 = column_without_suffix+'_1'
            local_df[new_column] = local_df[column_device_1]-local_df[column_device_2]
            local_df = local_df.drop(columns=[column_device_1, column_device_2])
    return local_df

In [ ]:
feature_device_based_df = drift_features(drift_filtered_df)
feature_device_based_df = calc_device_diffs(feature_device_based_df,feature_device_based_df.columns[5:])
feature_device_based_df.head()

In [ ]:
drift_feature_df = feature_device_based_df.merge(drift_processed_df, on=['age_group', 'subject', 'uuid'])
drift_feature_df.head()

In [ ]:
correlation_matrix(drift_feature_df)

In [ ]:
drift_feature_df = drift_feature_df.drop(columns=['x_sem','y_sem','z_sem','mag_sem','x_mean','mag_mean','mag_snr','y_snr','x_peaks','y_peaks','z_peaks','mag_snr_diff','y_snr_diff','x_mean_diff','mag_mean_diff'])

In [ ]:
plot_feature_columns(drift_feature_df,'std')
plot_feature_columns(drift_feature_df,'sal')
box_plot_columns(drift_feature_df, 'age_group', ['y_mean','z_mean'])
box_plot_columns(drift_feature_df, 'age_group', ['x_snr','z_snr'])
box_plot_columns(drift_feature_df, 'age_group', ['mag_peaks'])
box_plot_columns(drift_feature_df, 'age_group', ['x_std_diff','y_std_diff','z_std_diff','mag_std_diff'])
box_plot_columns(drift_feature_df, 'age_group', ['y_mean_diff','z_mean_diff'])
box_plot_columns(drift_feature_df, 'age_group', ['x_peaks_diff','y_peaks_diff','z_peaks_diff','mag_peaks_diff'])
box_plot_columns(drift_feature_df, 'age_group', ['x_snr_diff','z_snr_diff'])
box_plot_columns(drift_feature_df, 'age_group', ['x_sal_diff','y_sal_diff','z_sal_diff','mag_sal_diff'])

# ML Models

In [ ]:
results = {}

In [ ]:
feature_keys = ['z_std','z_snr','y_std_diff','z_std_diff','y_mean_diff','y_peaks_diff','x_snr_diff','y_sal_diff']
class_key = ['age_group']

In [ ]:
drift_feature_df = drift_feature_df.sort_values(by='age_group')
for feature in feature_keys:
    box_plot_columns(drift_feature_df, 'subject',[feature],show_column_in_label=False)

In [ ]:
drift_feature_df['age_group'].replace(to_replace=30,value=1,inplace=True)
drift_feature_df['age_group'].replace(to_replace=50,value=0,inplace=True)

## Principal Component Analysis

In [ ]:
principal_components_df = principal_component_analysis(drift_feature_df, feature_keys)

In [ ]:
plot_principal_component_analysis(drift_feature_df, principal_components_df.reset_index(drop=False), 'age_group', [30,50])

## Run Algorithms on extracted Features

In [ ]:
feature_results = run_feature_algorithms(drift_feature_df, feature_keys)
results.update(feature_results)

## Time Series Pre-Processing

In [ ]:
def extract_sequences_by_device(session_df):
    device_sequences = {}
    device_sequences_length = {}
    for device in session_df['device'].unique():
        device_df = session_df[session_df['device']==device]
        sequence = device_df[['x','y','z','mag']].values
        device_sequences[device] = sequence
        device_sequences_length[device] = len(sequence)
    return device_sequences, device_sequences_length


In [ ]:
def build_session_sequence(device_sequences, device_sequences_length):
    final_sequence = []
    min_length = min(device_sequences_length.values())
    for device in device_sequences.keys():
        device_sequence = device_sequences[device]
        for index in range(min_length):
            device_sequence_entry = device_sequence[index]
            if len(final_sequence) <= index:
                final_sequence.append(device_sequence_entry)
            else:
                current_sequence_entry = final_sequence[index]
                new_sequence_entry = list(np.append(current_sequence_entry,device_sequence_entry))
                final_sequence[index] = new_sequence_entry
    return final_sequence

In [ ]:
def extract_sequences_and_labels(df):
    sequences = []
    labels = []
    length = []
    for uuid in df['uuid'].unique():
        current_df = df[df['uuid']==uuid]
        labels.append(current_df['age_group'].max())
        device_sequences, device_sequences_length = extract_sequences_by_device(current_df)
        session_sequence = build_session_sequence(device_sequences, device_sequences_length)
        sequences.append(session_sequence)
        length.append(len(session_sequence))
    target_length = np.int_(pd.Series(length).quantile(0.65))
    final_sequences = align_sequences_to_same_length(sequences,target_length)
    return np.stack(final_sequences), labels

## Run Time Series Algorithm

In [ ]:
time_series_results = run_time_series_algorithms(drift_df, compile_sequences_function=extract_sequences_and_labels)
results.update(time_series_results)

## ML Results

In [ ]:
results